In [1]:
notebook_name = '71_SHAP_28_nb68_fold2.ipynb'

#元notebook_name = "68_鋳造製品_EfficientNet-B0_106.ipynb"
#MultiTaskEfficientNet_FPN
#segmentation_class_weights = {0: 51, 1: 85, 2: 55, 3: 22, 4: 1, 5: 80, 6: 18}
# 0: 黒い点　1: 傷　2: 白いへこみ　3: 縁のへこみ　4: はね　5: 段上の傷　6: 複雑な中心が暗いへこみ
#weight=2.0
# 交差検証

# ライブラリ用意

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install shap

In [4]:
import sys
import os
import re
import random
from pathlib import Path
from glob import glob
from PIL import Image
import torch
from torch import nn
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import shap
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
sys.path.append(os.path.abspath('/content/drive/Othercomputers/マイ パソコン/Defect_detection_in_cast_products/src'))
sys.path.append(os.path.abspath('/content/drive/Othercomputers/マイ パソコン/Defect_detection_in_cast_products/outputs'))

In [6]:
print(sys.path)

['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/tmp/tmp6ouxb98g', '/content/drive/Othercomputers/マイ パソコン/Defect_detection_in_cast_products/src', '/content/drive/Othercomputers/マイ パソコン/Defect_detection_in_cast_products/outputs']


In [7]:
from MultiTaskEfficientNet_FPN import MultiTaskEfficientNet_FPN
from set_seed import set_seed
from path_config import RAW_TRAIN_DIR, RAW_TEST_DIR, MODEL_DIR, SHAP_DIR

In [8]:
set_seed(42)

# 各ワーカーのシード固定　DataLoaderの引数に渡す
def seed_worker(worker_id):
    worker_seed = 42 + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# 乱数生成器
g = torch.Generator()
g.manual_seed(42)

In [9]:
SHAP_notebook_number = re.search(r'_(\d+)_', notebook_name).group(1)
print(SHAP_notebook_number)

28


# モデルの用意

In [10]:
main_notebook_number = re.search(r'nb(\d+)', notebook_name).group(1)
print(main_notebook_number)
fold_number = re.search(r'fold(\d+)', notebook_name).group(1)
print(fold_number)
NET_PATH = MODEL_DIR / f'model_fold{fold_number}_nb{main_notebook_number}.pth'

68
2


In [11]:
# モデルの読み込み
net = MultiTaskEfficientNet_FPN()
net.load_state_dict(torch.load(NET_PATH, map_location='cuda'))
net.eval().cuda()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 139MB/s] 


MultiTaskEfficientNet_FPN(
  (stem): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
  )
  (block1): Sequential(
    (0): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
       

In [12]:
# ラッパークラス定義
class ClassifierOnlyWrapper(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model

    def forward(self, x):
        class_output, _ = self.base(x)
        return torch.nn.functional.softmax(class_output, dim=1)

# ラップしてExplainerに渡す
wrapped_model = ClassifierOnlyWrapper(net)

# 画像用意

In [13]:
# 前処理の定義
transform = A.Compose([
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.Resize(224, 224),
    A.ColorJitter(brightness=0.1, contrast=0.1, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], additional_targets={'mask': 'mask'})

In [14]:
# テスト画像（前処理前）をリストにする
test_image_paths = sorted(glob(str(Path(RAW_TEST_DIR) / '*.jpeg')))  # 画像パス
raw_images = [Image.open(p).convert('RGB') for p in test_image_paths]  # PIL画像
input_tensors = torch.stack([transform(image=np.array(img))['image'] for img in raw_images]).to("cuda")  # Tensor化

In [15]:
background_image_paths = sorted(glob(str(Path(RAW_TRAIN_DIR) / '*.jpeg')))  # 画像パス
raw_background_images = [Image.open(p).convert('RGB') for p in background_image_paths[-10:-1]]  # PIL画像 最後の10枚を使う
background_input_tensors = torch.stack([transform(image=np.array(img))['image'] for img in raw_background_images]).to("cuda")  # Tensor化

# 基準画像
background = background_input_tensors[:5]

In [16]:
# 入力画像の正規化を戻す（可視化のため）
def unnormalize(img_tensor, mean, std):
    img = img_tensor.clone().detach().cpu()
    for c in range(3):
        img[c] = img[c] * std[c] + mean[c]
    return img

# SHAP実行

In [17]:
explainer = shap.GradientExplainer(wrapped_model, background)

In [18]:
SAVE_FOLDER = SHAP_DIR / "_".join(Path(notebook_name).stem.split("_")[1:])  # "SHAP_17_nb21"

In [19]:
# RGBを合計
for i in range(len(input_tensors)):

    # input_tensors[0]: 画像、test_image: 説明対象、background: 基準画像）
    test_image = input_tensors[i].unsqueeze(0)  # 1枚目を使って可視化（[1,3,224,224]）

    # 予測クラスの取得（softmaxしてargmax）
    pred_class = wrapped_model.forward(test_image).argmax(dim=1).item()

    if pred_class == 0:

        unnorm_image = unnormalize(test_image[0], [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        unnorm_image = unnorm_image.permute(1, 2, 0).numpy()  # CHW → HWC

        # SHAP値の計算
        shap_values = explainer.shap_values(test_image) # [(1, 3, 224, 224), (1, 3, 224, 224)]

        # ---- SHAP出力（クラス数分のリスト）から０クラス分を取り出す ----
        shap_vals = shap_values[0]  #クラス0(欠陥)のSHAP値: shape

        # ---- SHAP値を画像形式に変換 ----
        shap_img = shap_vals[0]  # 1枚目の画像 shape = (224, 224, 2)
        #shap_map = np.abs(shap_img).mean(axis=-1)  # RGBの平均をとる → shape=(224,224)
        shap_map = np.abs(shap_img).sum(axis=-1)  # RGBを合計
        shap_map = shap_map / shap_map.max()  # 正規化
        shap_map = np.nan_to_num(shap_map)              # NaN対策

        # ファイル名取得
        img_filename = Path(test_image_paths[i]).name

        # 可視化
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(unnorm_image)
        plt.imshow(shap_map, cmap='jet', alpha=0.5)  # alphaで透明度調整
        plt.axis('off')
        plt.title(f'SHAP heatmap\n{img_filename}')

        plt.subplot(1, 2, 2)
        plt.imshow(unnorm_image)
        plt.axis('off')
        plt.title(f'Original Image\n{img_filename}')
        plt.tight_layout()

        # ファイルに保存
        os.makedirs(SAVE_FOLDER, exist_ok=True)
        shap_img_save_path = SAVE_FOLDER / f'SHAP_{SHAP_notebook_number}_{i}.png'
        plt.savefig(shap_img_save_path)

        plt.show()

Output hidden; open in https://colab.research.google.com to view.